In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, basename
import pickle
from scipy.stats import pearsonr
from scipy.signal import periodogram, filtfilt, spectrogram, welch
import seaborn as sns

from filtering import freq_filt

banner_coords = (44.3, -115.233)
res = pd.read_csv('/bsuscratch/zacharykeskinen/data/infrasound/eq_catalog/selected_v2.csv')
from shapely import wkt
res['geometry'] = res['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(res, geometry = 'geometry', crs = 'EPSG:4326')

result_dir = '/bsuscratch/zacharykeskinen/data/infrasound/psd_results'
data_dir = '/bsuscratch/zacharykeskinen/data/infrasound/array_data'
with open(join(data_dir, 'merged/all_days'), 'rb') as f:
    days = pickle.load(f)
sps = 200

In [18]:
import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [21]:
n = 0
avg_Pxx = np.array([])
all_psds = {}
for i, r in res[res.selected == 1].iterrows():
    if i < 250:
        dt = pd.to_datetime(r.time).strftime('%Y-%m-%d')
        day = days[dt]
        sig = {}
        s = pd.to_datetime(r.time) + pd.Timedelta('3 second')
        e = s + pd.Timedelta('10 second')
        try:
            if 0.33 in day.keys() and 1.33 in day.keys() and day['snotel']['Snow Depth (cm) Start of Day Values'] > 133:
                for name, fp in day.items():
                    if name != 'snotel':
                        arr = freq_filt(pd.read_parquet(fp)[s:e].values.ravel(), 1, kind = 'highpass')
                        arr = arr[:2000]
                        f, Pxx = welch(arr, sps, scaling = 'density', window = 'hann')
                        # Pxx = filtfilt([1,1,1,1,1],5, Pxx)
                        sig[name] = Pxx
                df = pd.DataFrame(sig)
                df.index = f
                if avg_Pxx.size == 0:
                    avg_Pxx = df
                else:
                    avg_Pxx = avg_Pxx + df
                    n +=1
                all_psds[r.time] = df
        except ValueError as e:
            print(dt)
            print(e)

avg_Pxx = avg_Pxx/n

Elapsed time: 0.000002 seconds.

Elapsed time: 0.000001 seconds.

Elapsed time: 0.000002 seconds.

Elapsed time: 0.000001 seconds.

Elapsed time: 0.000001 seconds.

Elapsed time: 0.000002 seconds.

Elapsed time: 0.000001 seconds.

Elapsed time: 0.000001 seconds.

Elapsed time: 0.000001 seconds.

Elapsed time: 0.000019 seconds.

Elapsed time: 0.000014 seconds.

Elapsed time: 0.000014 seconds.

Elapsed time: 0.000013 seconds.

Elapsed time: 7.798862 seconds.

2022-03-20
The length of the input vector x must be greater than padlen, which is 9.
Elapsed time: 7.775490 seconds.

Elapsed time: 10.698168 seconds.

Elapsed time: 7.680823 seconds.

Elapsed time: 7.622433 seconds.

Elapsed time: 7.887062 seconds.

Elapsed time: 10.490912 seconds.

Elapsed time: 7.619695 seconds.

Elapsed time: 7.814464 seconds.

Elapsed time: 11.885744 seconds.

Elapsed time: 10.951961 seconds.

Elapsed time: 7.803420 seconds.

Elapsed time: 11.319084 seconds.

Elapsed time: 7.783036 seconds.

Elapsed time: 7.881

In [23]:
avg_Pxx

,0.33,0.66,1.00,1.33,2.00
0.00000,1.846384e-06,2.057392e-06,3.648032e-06,6.165037e-06,4.504637e-04
0.78125,7.238740e-06,8.489053e-06,1.415228e-05,2.499817e-05,3.431961e-03
1.56250,9.986950e-06,1.051671e-05,1.369896e-05,2.319259e-05,1.275675e-02
2.34375,8.277388e-06,8.552071e-06,7.641857e-06,9.227289e-06,7.891541e-03
3.12500,5.165071e-06,5.228629e-06,4.616108e-06,4.830767e-06,3.028237e-03
...,...,...,...,...,...
96.87500,9.692498e-12,1.127637e-11,9.819289e-12,1.096596e-11,1.060082e-11
97.65625,9.330073e-12,1.119627e-11,1.003376e-11,9.789820e-12,1.021388e-11
98.43750,9.963827e-12,8.934213e-12,8.975083e-12,9.395030e-12,9.948429e-12
99.21875,9.421039e-12,8.524337e-12,8.272235e-12,9.769778e-12,9.814928e-12


In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, basename
import pickle
from scipy.stats import pearsonr
from scipy.signal import periodogram, filtfilt, spectrogram, welch
import seaborn as sns

from filtering import freq_filt

banner_coords = (44.3, -115.233)
res = pd.read_csv('/bsuscratch/zacharykeskinen/data/infrasound/eq_catalog/selected_v2.csv')
from shapely import wkt
res['geometry'] = res['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(res, geometry = 'geometry', crs = 'EPSG:4326')

result_dir = '/bsuscratch/zacharykeskinen/data/infrasound/psd_results'
data_dir = '/bsuscratch/zacharykeskinen/data/infrasound/array_data'
with open(join(data_dir, 'merged/all_days'), 'rb') as f:
    days = pickle.load(f)
sps = 200

############################### Welch #####################################
# Get welch earhtquake averages
window_start = [0, 3, 5, 10]
windows_end = [5,10,15,30]
for start in window_start:
    for end in windows_end:
        if start >= end:
            print('nope')
            break
        n = 0
        avg_Pxx = np.array([])
        for i, r in res[res.selected == 1].iterrows():
            if i == 0:
                dt = pd.to_datetime(r.time).strftime('%Y-%m-%d')
                day = days[dt]
                sig = {}
                s = pd.to_datetime(r.time) + pd.Timedelta(f'{start} second')
                e = s + pd.Timedelta(f'{end} second')
        #         try:
        #             if 0.33 in day.keys() and 1.33 in day.keys() and day['snotel']['Snow Depth (cm) Start of Day Values'] > 133:
        #                 for name, fp in day.items():
        #                     if name != 'snotel':
        #                         arr = freq_filt(pd.read_parquet(fp)[s:e].values.ravel(), 1, kind = 'highpass')
        #                         arr = arr[:2000]
        #                         f, Pxx = welch(arr, sps, scaling = 'density', window = 'hann')
        #                         # Pxx = filtfilt([1,1,1,1,1],5, Pxx)
        #                         sig[name] = Pxx
        #                 df = pd.DataFrame(sig)
        #                 df.index = f
        #                 if avg_Pxx.size == 0:
        #                     avg_Pxx = df
        #                 else:
        #                     avg_Pxx = avg_Pxx + df
        #                     n +=1
        #         except ValueError as e:
        #             print(dt)
        #             print(e)

        # avg_Pxx = avg_Pxx/n

        # with open(join(result_dir, 'avg_welch.pkl'), 'wb') as f:
        #     pickle.dump(avg_Pxx, f)

nope
nope
